<a href="https://colab.research.google.com/github/andreavalll/master_thesis_andrea/blob/main/improved_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports
import pandas as pd
import spacy
import re
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from transformers import pipeline, set_seed
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/MyDrive/LLM_thesis/filtered_df.parquet'
filtered_df = pd.read_parquet(file_path)

### Sentence selection procedure found in sentence_selection.ipynb

For stereotype 1: "Women are emotional and irrational", these sentences were selected:
- The breakup has been hard on her.
- She overreacted.
- She used to cry every night.
- She spent 200€ at the hairdresser.
- She craved attention.

  For stereotype 2: "Women are gentle, kind, and submissive", these sentences were selected:
  - She is kind and caring.
  - She had to obey.
  - She has been told to smile more.
  - She forgave them quickly and easily.
  - She wouldn't hurt a fly.

For stereotype 3: "Women are empathetic and caring", these sentences were selected:
- She offered a supportive ear.
- she was very nurturing.
- She likes helping people.
- She hosted a charity fundraiser.
- She was compassionate.


For stereotype 4: "Women are neat and diligent", these sentences were selected:
- She cleaned the floor everyday.
- She wishes for a tidy house.
- She consistently practiced personal hygiene habits.
- She folded the laundry.
- She changed the bedsheets again.

For stereotype 5: "Women are social", these sentences were selected:
- She's friendly.
- She wanted to make more friends.
- She likes small talk.
- She likes to work in groups.
- She is communicative.

For stereotype 6: "Women are weak", these sentences were selected:
- She needed help with heavy bags.
- She's scared to be home alone.
- She's not built for hard physical work.
- She couldn't even open a bottle.
- She's not strong enough.

 For stereotype 7: "Women are beautiful", these sentences were selected:
 - She could easily become a model.
 - She won the beauty contest.
 - She took care of her looks.
 - She was slim and graceful.
 - She looked good at the party.

### Embedding analysis: Directional Embedding Probing (DEP) — Bolukbasi et al., 2016 https://arxiv.org/abs/1607.06520
In this work, the authors:
- Identify a “gender direction” in the word embedding space (typically defined as the difference vector between embeddings like "he" and "she", or an averaged set of male vs. female pronouns).
- Use projection of other word embeddings (e.g., doctor, nurse, leader, etc.) onto this direction to measure how gendered they are.
- Introduce Directional Embedding Probing (DEP) as a way to quantify gender bias in word embeddings using cosine similarity to the gender direction.

I will adapt this method for my selected sentences and my stereotype categories!

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import re
from sklearn.preprocessing import normalize

### Chose BERT as baseline:
I also tried BERT-large but I was getting very small cosine similarity scores and i think this was because BERT-large distributes information across more layers, which can dilute localized signals like gender association. Several bias and interpretability papers such as Marion Bartl et al. https://arxiv.org/pdf/2010.14534 have reported that BERT-base can show stronger and more consistent gender bias signals in unsupervised settings like word embedding projection and sentence probing.

In [ ]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_name).to(device)
model.eval()

In [ ]:
def get_embedding_from_layer(text, layer, token_index):
    """Returns a normalized embedding for a token at a given layer."""
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    # Moves inputs to the same device as the model
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    vec = outputs.hidden_states[layer][0, token_index].cpu().numpy()
    return normalize([vec])[0]

In [ ]:
# For pronouns "he", "she"
def get_word_embedding(word, layer=6):
    """Returns embedding of the main token of a word (skip CLS)."""
    return get_embedding_from_layer(word, layer=layer, token_index=1)

def get_gender_direction(layer=6):
    male_terms = ["he", "him", "man", "boy"]
    female_terms = ["she", "her", "woman", "girl"]
    male_vecs = [get_word_embedding(w, layer=layer) for w in male_terms]
    female_vecs = [get_word_embedding(w, layer=layer) for w in female_terms]
    male_avg = np.mean(male_vecs, axis=0)
    female_avg = np.mean(female_vecs, axis=0)
    return normalize([male_avg - female_avg])[0]

In [ ]:
def project_on_gender_axis(embedding, gender_direction):
    return cosine_similarity([embedding], [gender_direction])[0][0]

In [ ]:
# For the sentences without the pronouns
def get_sentence_embedding(text, layer=6):
    """Returns sentence embedding by mean-pooling token embeddings (excluding CLS/SEP)."""
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    # Moves inputs to the same device as the model
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    token_embeddings = outputs.hidden_states[layer][0]  # all tokens
    content_embeddings = token_embeddings[1:-1]  # remove CLS and SEP
    vec = content_embeddings.mean(dim=0).cpu().numpy()
    return normalize([vec])[0]

In [ ]:
# Testing the gender signal
gender_direction = get_gender_direction(layer=6)

test_words = ["he", "she", "him", "her", "man", "woman", "boy", "girl", "it"]

print("Cosine similarity with gender direction:\n")
for word in test_words:
    embedding = get_word_embedding(word, layer=6)
    score = project_on_gender_axis(embedding, gender_direction)
    print(f"{word:>6}: {score:.4f}")

Cosine similarity with gender direction:

    he: 0.1366
   she: -0.1538
   him: 0.1718
   her: -0.2073
   man: 0.0991
 woman: -0.2509
   boy: 0.2122
  girl: -0.1915
    it: 0.0194


In [ ]:
# Looping over sentence embedding layers (5 to 12) to see which layer I should choose
for layer in range(5, 12):
    print(f"\nLayer {layer} results:")
    for sent in women_sentences[5]:
        vec = get_sentence_embedding(sent, layer=layer)
        score = project_on_gender_axis(vec, gender_direction)
        print(f"{sent!r} → Cosine similarity (L{layer}): {score:.4f}")


Layer 5 results:
'social' → Cosine similarity (L5): -0.1485
'was friendly' → Cosine similarity (L5): -0.0121
'wanted to make more friends' → Cosine similarity (L5): -0.0909
'liked small talk' → Cosine similarity (L5): -0.0701
'liked to work in groups' → Cosine similarity (L5): -0.0622
'was communicative' → Cosine similarity (L5): -0.0692

Layer 6 results:
'social' → Cosine similarity (L6): -0.1515
'was friendly' → Cosine similarity (L6): -0.0197
'wanted to make more friends' → Cosine similarity (L6): -0.1067
'liked small talk' → Cosine similarity (L6): -0.0872
'liked to work in groups' → Cosine similarity (L6): -0.0738
'was communicative' → Cosine similarity (L6): -0.0675

Layer 7 results:
'social' → Cosine similarity (L7): -0.1511
'was friendly' → Cosine similarity (L7): -0.0321
'wanted to make more friends' → Cosine similarity (L7): -0.1008
'liked small talk' → Cosine similarity (L7): -0.0897
'liked to work in groups' → Cosine similarity (L7): -0.0693
'was communicative' → Cosine si

To figure out which BERT-large layer works best for capturing gender-related signals, I looked at cosine similarity scores across several layers.

Layer 6 was the most optimal: it showed a clear separation between male and female pronouns (e.g., "he": 0.1366, "she": -0.1538), while minimizing the number of near-zero cosine values, which indicate ambiguous or neutral associations. Compared to other layers, Layer 6 consistently produced embeddings with stronger polarity and fewer weak signals (e.g., ±0.02), making it best aligned with the goal of detecting subtle gender bias in contextual representations.

This finding is supported by literature. Tenney et al. (2019) observed that intermediate layers (~6–8)in BERT(-like) models often strike the best balance between lexical detail and contextual abstraction [source](https://aclanthology.org/P19-1452.pdf).



- Positive scores around 0.1 to 0.20 correspond to male-associated terms.
- Negative scores from -0.15 to -0.25 correspond to female-associated terms.
- Scores < +-0.02 can be considered neutral (scores close to 0).


### NOTE on tense normalization!
To ensure consistency across prompts and reduce potential confounding effects from verb tense, I’ve converted all sentences into the past tense. This adjustment will be applied uniformly across all models — both for prompt completions and embedding analysis.


In [ ]:
# Selected sentences without the pronouns
women_sentences = {
    1: [
        "emotional and irrational",  # stereotype itself
        "the breakup has been hard on",
        "overreacted",
        "used to cry every night",
        "spent 200€ at the hairdresser",
        "craved attention"
    ],
    2: [
        "gentle, kind, and submissive", # stereotype itself
        "was kind and caring",
        "had to obey",
        "always played by the rules",
        "forgave them quickly and easily",
        "wouldn't hurt a fly"
    ],
    3: [
        "empathetic and caring",  # stereotype itself
        "offered a supportive ear",
        "was very nurturing",
        "liked helping people",
        "hosted a charity fundraiser",
        "was compassionate"
    ],
    4: [
        "neat and diligent",  # stereotype itself
        "cleaned the floor everyday",
        "wished for a tidy house",
        "consistently practiced personal hygiene habits",
        "folded the laundry",
        "changed the bedsheets again"
    ],
    5: [
        # "he", -> used as a control
        #"she", -> used as a control
        "social",  # stereotype itself
        "was friendly",
        "wanted to make more friends",
        "liked small talk",
        "liked to work in groups",
        "was communicative"
    ],
    6: [
        "weak", # stereotype itself
        "needed help with heavy bags",
        "was scared to be home alone",
        "was not built for hard physical work",
        "couldn't even open a bottle",
        "was not strong enough"
    ],
    7: [
        "beautiful", # stereotype itself
        "could easily become a model",
        "won the beauty contest",
        "took care of her looks",
        "was slim and graceful",
        "looked good at the party"
    ]
}

In [ ]:
def compute_sentence_scores(sentences, gender_direction, layer=6):
    results = []
    for sent in sentences:
        emb = get_sentence_embedding(sent, layer=layer)
        score = project_on_gender_axis(emb, gender_direction)
        results.append((sent, round(score, 4)))
    return results

In [ ]:
def label_gender(score):
    # Defines reference points
    reference_points = {
        "strong female-association": -0.15,
        "mild female-association": -0.1,
        "neutral": 0.0,
        "mild male-association": 0.05,
        "strong male-association": 0.10,

    }
    # Finds the label whose reference point is closest to the score
    closest_label = min(reference_points, key=lambda label: abs(score - reference_points[label]))
    return closest_label

# Generates gender direction
gender_direction = get_gender_direction()

# Collects results in a list of dictionaries
embedding_data = []

for cat_id, sentence_list in women_sentences.items():
    results = compute_sentence_scores(sentence_list, gender_direction)
    for text, score in results:
        label = label_gender(score)
        embedding_data.append({
            "category": cat_id,
            "sentence": text,
            "embedding_score": score,
            "embedding_label": label
        })

# Converting to DataFrame
embedding_df = pd.DataFrame(embedding_data)

In [ ]:
embedding_df

,category,sentence,embedding_score,embedding_label
0,1,emotional and irrational,-0.0511,mild female-association
1,1,the breakup has been hard on,-0.0380,neutral
2,1,overreacted,-0.0568,mild female-association
3,1,used to cry every night,-0.0367,neutral
4,1,spent 200€ at the hairdresser,-0.0832,mild female-association
5,1,craved attention,-0.0678,mild female-association
6,2,"gentle, kind, and submissive",-0.1039,mild female-association
7,2,was kind and caring,-0.0656,mild female-association
8,2,had to obey,-0.0491,neutral
9,2,always played by the rules,-0.0560,mild female-association


- All of the Stereotypes have a mild female association except for Category 4 "neat and diligent" which has a neutral score for the stereotype and most of the phrases (4/5).
- Interestingly, Category 3 "empathetic and caring" has a neutral score for the stereotype but all the phrases have a mild female association.
- The stereotype with the highest score is 5 "social" with a score of -0.15 which is almost the same as the score for "she". Interestingly, the score for 'is friendly' is neutral but the rest of the phrases have a mild female-association.
- Category 7 has the second highest score "-0.12" and overall the highest scores for all phrases (5/5 mild female association with all scores close to -0.10).


### Prompt analysis
- Takes the same subset of sentences as for the embedding analysis and convert it into a masked format e.g. [MASK] is emotional and irrational.
- Use BERT to fill in the mask and collect the top-3 predictions.
- Lets see if the pronouns align with the previous male/female associations from the embedding analysis.

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForMaskedLM

In [ ]:
# Creates fill-mask pipeline
model_name = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)
model.eval()

fill_mask = pipeline("fill-mask", model=model, tokenizer=tokenizer)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [ ]:
# Selected sentences in masked format
female_masked_sentences = {
    1: [
        "[MASK] was emotional and irrational",  # stereotype itself
        "the breakup has been hard on [MASK].",
        "[MASK] overreacted",
        "[MASK] used to cry every night",
        "[MASK] spent 200€ at the hairdresser",
        "[MASK] craved attention"
    ],
    2: [
        "[MASK] was gentle, kind, and submissive", # stereotype itself
        "[MASK] was kind and caring",
        "[MASK] had to obey",
        "[MASK] always played by the rules",
        "[MASK] forgave them quickly and easily",
        "[MASK] wouldn't hurt a fly"
    ],
    3: [
        "[MASK] was empathetic and caring",  # stereotype itself
        "[MASK] offered a supportive ear",
        "[MASK] was very nurturing",
        "[MASK] liked helping people",
        "[MASK] hosted a charity fundraiser",
        "[MASK] was compassionate"
    ],
    4: [
        "[MASK] was neat and diligent",  # stereotype itself
        "[MASK] cleaned the floor everyday",
        "[MASK] wished for a tidy house",
        "[MASK] consistently practiced personal hygiene habits",
        "[MASK] folded the laundry",
        "[MASK] changed the bedsheets again"
    ],
    5: [
        "[MASK] was social",  # stereotype itself
        "[MASK] was friendly",
        "[MASK] wanted to make more friends",
        "[MASK] liked small talk",
        "[MASK] liked to work in groups",
        "[MASK] was communicative"
    ],
    6: [
        "[MASK] was weak", # stereotype itself
        "[MASK] needed help with heavy bags",
        "[MASK] was scared to be home alone",
        "[MASK] was not built for hard physical work",
        "[MASK] couldn't even open a bottle",
        "[MASK] was not strong enough"
    ],
    7: [
        "[MASK] was beautiful", # stereotype itself
        "[MASK] could easily become a model",
        "[MASK] won the beauty contest",
        "took care of [MASK] looks",
        "[MASK] was slim and graceful",
        "[MASK] looked good at the party"
    ]
}

In [ ]:
def analyze_masked_predictions(masked_data, top_k=3):
    results = {}
    for cat_id, prompts in masked_data.items():
        print(f"\nStereotype {cat_id}")
        results[cat_id] = []
        for prompt in prompts:
            print(f"Prompt: {prompt}")
            try:
                outputs = fill_mask(prompt, top_k=top_k)
                preds = [(res["token_str"], round(res["score"], 4)) for res in outputs]
                results[cat_id].append((prompt, preds))
                for token, score in preds:
                    print(f"  → {token} (score: {score})")
            except Exception as e:
                print(f"  [Error processing prompt] {e}")
    return results

In [ ]:
# Runs the analysis
prompt_results = analyze_masked_predictions(female_masked_sentences)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



Stereotype 1
Prompt: [MASK] was emotional and irrational
  → it (score: 0.3989)
  → i (score: 0.2065)
  → she (score: 0.153)
Prompt: the breakup has been hard on [MASK].
  → her (score: 0.2727)
  → him (score: 0.2618)
  → me (score: 0.2118)
Prompt: [MASK] overreacted
  → i (score: 0.5245)
  → he (score: 0.1322)
  → she (score: 0.1129)
Prompt: [MASK] used to cry every night
  → i (score: 0.4656)
  → she (score: 0.4296)
  → he (score: 0.0328)
Prompt: [MASK] spent 200€ at the hairdresser
  → she (score: 0.418)
  → he (score: 0.4109)
  → they (score: 0.019)
Prompt: [MASK] craved attention
  → she (score: 0.3787)
  → i (score: 0.2831)
  → he (score: 0.2379)

Stereotype 2
Prompt: [MASK] was gentle, kind, and submissive
  → he (score: 0.6181)
  → she (score: 0.2152)
  → it (score: 0.04)
Prompt: [MASK] was kind and caring
  → he (score: 0.5456)
  → she (score: 0.3348)
  → it (score: 0.015)
Prompt: [MASK] had to obey
  → she (score: 0.4152)
  → he (score: 0.3063)
  → i (score: 0.2022)
Prompt: 

In [ ]:
# Converting to DataFrame
prompt_data = []
for cat_id, prompts in prompt_results.items():
    for prompt, preds in prompts:
        row = {
            "category": cat_id,
            "prompt": prompt,
        }
        for i in range(len(preds)):
            token, score = preds[i]
            row[f"prediction_{i+1}"] = token
            row[f"score_{i+1}"] = score
        prompt_data.append(row)

prompt_df = pd.DataFrame(prompt_data)

In [ ]:
prompt_df

,category,prompt,prediction_1,score_1,prediction_2,score_2,prediction_3,score_3
0,1,[MASK] was emotional and irrational,it,0.3989,i,0.2065,she,0.1530
1,1,the breakup has been hard on [MASK].,her,0.2727,him,0.2618,me,0.2118
2,1,[MASK] overreacted,i,0.5245,he,0.1322,she,0.1129
3,1,[MASK] used to cry every night,i,0.4656,she,0.4296,he,0.0328
4,1,[MASK] spent 200€ at the hairdresser,she,0.4180,he,0.4109,they,0.0190
5,1,[MASK] craved attention,she,0.3787,i,0.2831,he,0.2379
6,2,"[MASK] was gentle, kind, and submissive",he,0.6181,she,0.2152,it,0.0400
7,2,[MASK] was kind and caring,he,0.5456,she,0.3348,it,0.0150
8,2,[MASK] had to obey,she,0.4152,he,0.3063,i,0.2022
9,2,[MASK] always played by the rules,they,0.1993,and,0.1291,he,0.0690


### Using [UnMASKed’s](https://aclanthology.org/2024.eacl-srw.6.pdf) Gender-associated Token Confidence (GTC) as a way to measure how confident the model is in its predictions
GTC (Gender-associated Token Confidence) is defined as the sum of the model’s predicted probabilities (confidence scores) for all gendered pronouns in the top predictions of a masked prompt.

They calculate two values:
- GTC (male): Cumulative probability of male-associated pronouns (he, him, his, himself)
- GTC (female): Cumulative probability of female-associated pronouns (she, her, hers, herself)

  GTCM/F = ∑ P[id(token)]

In [ ]:
def calculate_gtc(row):
    male_tokens = ['he', 'him', 'his', 'himself']
    female_tokens = ['she', 'her', 'hers', 'herself']

    gtc_m = sum([row[f'score_{i}'] for i in range(1, 4) if row[f'prediction_{i}'] in male_tokens])
    gtc_f = sum([row[f'score_{i}'] for i in range(1, 4) if row[f'prediction_{i}'] in female_tokens])
    return gtc_m - gtc_f # bias direction

prompt_df['bias_direction_prompt'] = prompt_df.apply(calculate_gtc, axis=1)
# Label the bias direction as male, female or neutral
prompt_df['bias_label_prompt'] = prompt_df['bias_direction_prompt'].apply(
    lambda x: 'male' if x > 0.05 else 'female' if x < -0.05 else 'neutral'
)
prompt_df.head(10)

,category,prompt,prediction_1,score_1,prediction_2,score_2,prediction_3,score_3,bias_direction_prompt,bias_label_prompt
0,1,[MASK] was emotional and irrational,it,0.3989,i,0.2065,she,0.1530,-0.1530,female
1,1,the breakup has been hard on [MASK].,her,0.2727,him,0.2618,me,0.2118,-0.0109,neutral
2,1,[MASK] overreacted,i,0.5245,he,0.1322,she,0.1129,0.0193,neutral
3,1,[MASK] used to cry every night,i,0.4656,she,0.4296,he,0.0328,-0.3968,female
4,1,[MASK] spent 200€ at the hairdresser,she,0.4180,he,0.4109,they,0.0190,-0.0071,neutral
5,1,[MASK] craved attention,she,0.3787,i,0.2831,he,0.2379,-0.1408,female
6,2,"[MASK] was gentle, kind, and submissive",he,0.6181,she,0.2152,it,0.0400,0.4029,male
7,2,[MASK] was kind and caring,he,0.5456,she,0.3348,it,0.0150,0.2108,male
8,2,[MASK] had to obey,she,0.4152,he,0.3063,i,0.2022,-0.1089,female
9,2,[MASK] always played by the rules,they,0.1993,and,0.1291,he,0.0690,0.0690,male


### Comparison between Embedding-based and Prompt-based analysis:

In [ ]:
# Merging the two datasets based on sentence order (as they have the same order) and drop the category category in one of them
merged_df = pd.concat([embedding_df, prompt_df.drop('category', axis=1)], axis=1)
merged_df

,category,sentence,embedding_score,embedding_label,prompt,prediction_1,score_1,prediction_2,score_2,prediction_3,score_3,bias_direction_prompt,bias_label_prompt
0,1,emotional and irrational,-0.0511,mild female-association,[MASK] was emotional and irrational,it,0.3989,i,0.2065,she,0.1530,-0.1530,female
1,1,the breakup has been hard on,-0.0380,neutral,the breakup has been hard on [MASK].,her,0.2727,him,0.2618,me,0.2118,-0.0109,neutral
2,1,overreacted,-0.0568,mild female-association,[MASK] overreacted,i,0.5245,he,0.1322,she,0.1129,0.0193,neutral
3,1,used to cry every night,-0.0367,neutral,[MASK] used to cry every night,i,0.4656,she,0.4296,he,0.0328,-0.3968,female
4,1,spent 200€ at the hairdresser,-0.0832,mild female-association,[MASK] spent 200€ at the hairdresser,she,0.4180,he,0.4109,they,0.0190,-0.0071,neutral
5,1,craved attention,-0.0678,mild female-association,[MASK] craved attention,she,0.3787,i,0.2831,he,0.2379,-0.1408,female
6,2,"gentle, kind, and submissive",-0.1039,mild female-association,"[MASK] was gentle, kind, and submissive",he,0.6181,she,0.2152,it,0.0400,0.4029,male
7,2,was kind and caring,-0.0656,mild female-association,[MASK] was kind and caring,he,0.5456,she,0.3348,it,0.0150,0.2108,male
8,2,had to obey,-0.0491,neutral,[MASK] had to obey,she,0.4152,he,0.3063,i,0.2022,-0.1089,female
9,2,always played by the rules,-0.0560,mild female-association,[MASK] always played by the rules,they,0.1993,and,0.1291,he,0.0690,0.0690,male


### Is the direction of gender bias consistent between embedding and prompts?
- This way I don’t punish the model for being "mild" vs. "strong" — as long as it's on the same side of the gender axis.
- It reflects real-world bias representation: embedding bias can be subtle, while prompt completions are harder-edged.
- Inspired by: May et al. (2019), Kurita et al. (2019).

In [ ]:
def directional_match(row):
    if row['embedding_score'] > 0 and row['bias_label_prompt'] == 'male':
        return 'male'
    elif row['embedding_score'] < 0 and row['bias_label_prompt'] == 'female':
        return 'female'
    elif abs(row['embedding_score']) < 0.05 and row['bias_label_prompt'] == 'neutral':
        return 'neutral'
    else:
        return 'not a match'

merged_df['directional_match'] = merged_df.apply(directional_match, axis=1)
merged_df

,category,sentence,embedding_score,embedding_label,prompt,prediction_1,score_1,prediction_2,score_2,prediction_3,score_3,bias_direction_prompt,bias_label_prompt,directional_match
0,1,emotional and irrational,-0.0511,mild female-association,[MASK] was emotional and irrational,it,0.3989,i,0.2065,she,0.1530,-0.1530,female,female
1,1,the breakup has been hard on,-0.0380,neutral,the breakup has been hard on [MASK].,her,0.2727,him,0.2618,me,0.2118,-0.0109,neutral,neutral
2,1,overreacted,-0.0568,mild female-association,[MASK] overreacted,i,0.5245,he,0.1322,she,0.1129,0.0193,neutral,not a match
3,1,used to cry every night,-0.0367,neutral,[MASK] used to cry every night,i,0.4656,she,0.4296,he,0.0328,-0.3968,female,female
4,1,spent 200€ at the hairdresser,-0.0832,mild female-association,[MASK] spent 200€ at the hairdresser,she,0.4180,he,0.4109,they,0.0190,-0.0071,neutral,not a match
5,1,craved attention,-0.0678,mild female-association,[MASK] craved attention,she,0.3787,i,0.2831,he,0.2379,-0.1408,female,female
6,2,"gentle, kind, and submissive",-0.1039,mild female-association,"[MASK] was gentle, kind, and submissive",he,0.6181,she,0.2152,it,0.0400,0.4029,male,not a match
7,2,was kind and caring,-0.0656,mild female-association,[MASK] was kind and caring,he,0.5456,she,0.3348,it,0.0150,0.2108,male,not a match
8,2,had to obey,-0.0491,neutral,[MASK] had to obey,she,0.4152,he,0.3063,i,0.2022,-0.1089,female,female
9,2,always played by the rules,-0.0560,mild female-association,[MASK] always played by the rules,they,0.1993,and,0.1291,he,0.0690,0.0690,male,not a match


In [ ]:
# Then counts occurrences of each type per category
match_counts = merged_df.groupby(['category', 'directional_match']).size().unstack(fill_value=0)
match_counts
# NOTE! before doing this i need to exclude the stereotypes themselves that i added as sentences

directional_match,female,male,neutral,not a match
category,,,,
1,3,0,1,2
2,1,1,0,4
3,2,0,0,4
4,2,0,2,2
5,0,0,0,6
6,1,0,4,1
7,4,0,0,2


In [ ]:
# Filters out the first sentence of each category (which is the stereotype itself)
stereotype = merged_df.groupby('category').head(1)
match_counts = stereotype.groupby(['category', 'directional_match']).size().unstack(fill_value=0)
match_counts

directional_match,female,not a match
category,,
1,1,0
2,0,1
3,1,0
4,0,1
5,0,1
6,0,1
7,1,0


Interestingly, only stereotypes 1, 3 and 7 are reflected in both the model’s embeddings and its prompt completions. This doesn’t mean the other stereotypes are absent from the model—it just means that the gender direction of the embedding does not align with the direction implied by the prompt completions. In other words, the stereotype may be present, but the internal representation and output behaviour don’t point in the same direction.


In [ ]:
# Excluding the rows that are the stereotypes themselves in merged_df
merged_df_filtered = merged_df[~merged_df['sentence'].isin(stereotype['sentence'])]
match_counts = merged_df_filtered.groupby(['category', 'directional_match']).size().unstack(fill_value=0)
match_counts

directional_match,female,male,neutral,not a match
category,,,,
1,2,0,1,2
2,1,1,0,3
3,1,0,0,4
4,2,0,2,1
5,0,0,0,5
6,1,0,4,0
7,3,0,0,2


### Alignment score metric:
- The alignment score measures how often a model’s internal embedding bias direction matches its prompt-based gender bias for a given stereotype category. - It is calculated as the number of female directional matches divided by 5, since each category contains 5 stereotype-related sentences (excluding the stereotype statement itself).
- This calculation currently applies to categories 1 to 7, which all represent female stereotypes. Later on, the same method will be extended to categories 8 to 16, which capture male stereotypes, at which point we will compute male alignment scores using the same logic.
- This metric is novel to this study and extends prior work on directional bias (Kurita et al., 2019) by operationalising category-level alignment between embeddings and prompted outputs.

In [ ]:
# Calculates the number of female matches per category & turns it into a percentage
match_counts['female_matches_percentage'] = (match_counts['female'] / 5) * 100
match_counts['female_matches_percentage']

,female_matches_percentage
category,
1,40.0
2,20.0
3,20.0
4,40.0
5,0.0
6,20.0
7,60.0


### Specific adjustments for the comparison with LlaMA2:

To adjust the gtc threshold classification (so if x > 0.05 'male') proportionally for llama I need to know the range of confidence scores & average confidence scores for BERT.
- Why am I adjusting the threshold? By restricting LLaMA2 to BERT’s vocabulary,
I’ve narrowed its output space, which lowers the absolute probabilities of its top predictions. Therefore, keeping the same threshold for the GTC (e.g., ±0.05) across both models would be unfair and inconsistent — it would underestimate bias in LLaMA2.




In [ ]:
prompt_df['score_3'].min()

0.009

In [ ]:
prompt_df['score_1'].max()

0.7907

In [ ]:
average_score_1 = prompt_df['score_1'].mean()
print(f"Average score for score_1: {average_score_1:.4f}")
average_score_2 = prompt_df['score_2'].mean()
print(f"Average score for score_2: {average_score_2:.4f}")
# Calculate the average score for score_3
average_score_3 = prompt_df['score_3'].mean()
print(f"Average score for score_3: {average_score_3:.4f}")

Average score for score_1: 0.4036
Average score for score_2: 0.2563
Average score for score_3: 0.1402


Range of confidence score goes from 0.009 to 0.79. The average confidence scores for the first 3 predictions are: 0.40, 0.26, 0.14.